In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import tensorflow as tf
from sklearn.metrics import f1_score
tf.__version__

Using TensorFlow backend.


'1.15.0'

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub


In [3]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
train.head(3)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1


In [7]:
def clean_tweets(tweet):
    """Removes links and non-ASCII characters"""
    
    tweet = ''.join([x for x in tweet if x in string.printable])
    
    # Removing URLs
    tweet = re.sub(r"http\S+", "", tweet)
    
    return tweet

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punctuations(text):
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`"
    
    for p in punctuations:
        text = text.replace(p, f' {p} ')

    text = text.replace('...', ' ... ')
    
    if '...' not in text:
        text = text.replace('..', ' ... ')
    
    return text

def convert_abbrev(word):
    return abbreviations[word.lower()] if word.lower() in abbreviations.keys() else word

def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [convert_abbrev(word) for word in tokens]
    text = ' '.join(tokens)
    return text

In [8]:
ids_with_target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
train.at[train['id'].isin(ids_with_target_error),'target'] = 0
train[train['id'].isin(ids_with_target_error)]

train = train.drop(train[train["text"].duplicated()].index)

with open('../abbreviation.json') as json_file:
    abbreviations = json.load(json_file)

train["text"] = train["text"].apply(lambda x: clean_tweets(x))
test["text"] = test["text"].apply(lambda x: clean_tweets(x))

train["text"] = train["text"].apply(lambda x: remove_emoji(x))
test["text"] = test["text"].apply(lambda x: remove_emoji(x))

train["text"] = train["text"].apply(lambda x: remove_punctuations(x))
test["text"] = test["text"].apply(lambda x: remove_punctuations(x))

train["text"] = train["text"].apply(lambda x: convert_abbrev_in_text(x))
test["text"] = test["text"].apply(lambda x: convert_abbrev_in_text(x))

# ELMo

In [9]:
embed = hub.Module('https://tfhub.dev/google/elmo/3', trainable=True)

embeddings = embed(
    np.array(train["text"]),
    signature="default",
    as_dict=True)["default"]
embeddings

embeddings_test = embed(
    np.array(test["text"]),
    signature="default",
    as_dict=True)["default"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [10]:
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  sess.run(tf.tables_initializer())
  x_test = sess.run(embeddings_test)

In [13]:
pd.DataFrame(x_test).to_csv("../elmo_emb_test.csv")

In [15]:
emb_test = pd.read_csv('../elmo_emb_test.csv')
emb_test

emb_train = pd.read_csv('./elmo_emb.csv')

In [16]:
from sklearn.model_selection import train_test_split

#Need to clean "train[label]" for actual labels
xtrain, xvalid, ytrain, yvalid = train_test_split(emb_train, 
                                                  train["target"],  
                                                  random_state=42, 
                                                  test_size=0.15)
len_act = len(xtrain)

## Kneighbors

In [16]:
from sklearn.neighbors import KNeighborsClassifier


In [17]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(emb_train, train["target"])

full_predict_neigh = neigh.predict(emb_train)
f1_scor = f1_score(train["target"].values,full_predict_neigh,  average='micro')
f1_scor_2 = f1_score(train["target"].values, full_predict_neigh, average='weighted')
print(f1_scor, f1_scor_2)

0.8547247767559643 0.8540356762645799


In [9]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(xtrain, ytrain)

predict_neigh = neigh.predict(xvalid)
f1_scor = f1_score(yvalid, predict_neigh, average='micro')
f1_scor_2 = f1_score(yvalid, predict_neigh, average='weighted')

print(f1_scor, f1_scor_2)

0.7184724689165186 0.7182203029717853


In [18]:
full_predict_neigh

array([1, 1, 1, ..., 1, 1, 1])

## MLP

In [18]:
from sklearn.neural_network import MLPClassifier
from itertools import product

parameters = dict(
    layers = [49, 50, 51],
    iteration = [100, 300, 500],
    lrs = ["constant", "invscaling", "adaptive"]
)
param_values = [v for v in parameters.values()]

f1_hist = []
best_score = 0

mlp_clf = MLPClassifier(50, learning_rate = "constant", max_iter=100, random_state = 42)
mlp_clf.fit(xtrain, ytrain)

elmo_result_train = mlp_clf.predict(emb_train)
elmo_f1_sco_train = f1_score(elmo_result_train, train["target"].values)

full_predict_mlp = mlp_clf.predict(emb_train)
f1_scor_mlp = f1_score(train["target"].values,full_predict_mlp,  average='micro')
f1_scor_2_mlp = f1_score(train["target"].values, full_predict_mlp, average='weighted')

print(f1_scor_mlp, f1_scor_2_mlp)

0.851392776222844 0.8503097318927003


In [20]:
elmo_result_test = mlp_clf.predict_proba(emb_test)
pd.DataFrame(elmo_result_test).to_csv('../ELMo_test_proba.csv')

In [33]:
mlp_clf.classes_

array([0, 1])

In [29]:
pd.DataFrame(elmo_result_train)

,0,1
0,7.199286e-04,0.999280
1,3.309274e-07,1.000000
2,1.887770e-04,0.999811
3,6.089702e-07,0.999999
4,4.617704e-04,0.999538
5,2.315826e-07,1.000000
6,6.328398e-09,1.000000
7,1.193736e-03,0.998806
8,7.795002e-05,0.999922
9,3.591668e-05,0.999964


In [28]:
pd.DataFrame(elmo_result_train).to_csv('./ELMo_full_proba.csv')

In [18]:
mlp_clf = MLPClassifier(50, learning_rate = "constant", max_iter=100, random_state = 42)
mlp_clf.fit(xtrain, ytrain)

elmo_result_train = mlp_clf.predict(xtrain)
elmo_f1_sco_train = f1_score(elmo_result_train, ytrain)

elmo_result_valid = mlp_clf.predict(xvalid)
elmo_f1_sco_valid = f1_score(elmo_result_valid, yvalid)

print("f1 score train : ", elmo_f1_sco_train)
print("f1 score valid : ", elmo_f1_sco_valid)

f1 score train :  0.8251450676982592
f1 score valid :  0.7693989071038251


In [15]:
elmo_result_train

array([1, 1, 1, ..., 1, 1, 1])

In [73]:
emb_train = emb_train.drop("Unnamed: 0", axis = 1)

In [76]:
elmo_train = mlp_clf.predict(emb_train)
elmo_f1_train = f1_score(elmo_result_train, ytrain)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 18463 is different from 1024)

# TF IDF

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
list_corpus = train["text"].tolist()
list_labels = train["target"].tolist()

X_train, X_test, y_train, y_test = train_test_split(list_corpus, list_labels, test_size=0.2, 
                                                                                random_state=42)

In [27]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    print(np.shape(data))
    train = tfidf_vectorizer.fit_transform(data)
    print(np.shape(train))
    return train, tfidf_vectorizer

X_train_tfidf, tfidf_vectorizer = tfidf(X_train)
print(np.shape(X_test))
print(np.shape(y_train))

X_test_tfidf = tfidf_vectorizer.transform(X_test)

(6002,)
(6002, 18463)
(1501,)
(6002,)


In [28]:
X_train_tfidf

<6002x18463 sparse matrix of type '<class 'numpy.float64'>'
	with 87833 stored elements in Compressed Sparse Row format>

In [29]:
mlp_clf = MLPClassifier(30, learning_rate = "constant", max_iter=8, random_state = 42)
mlp_clf.fit(X_train_tfidf, y_train)

tfidf_result_train = mlp_clf.predict(X_train_tfidf)
tfidf_f1_sco_train = f1_score(tfidf_result_train, y_train)

tfidf_result_valid = mlp_clf.predict(X_test_tfidf)
tfidf_f1_sco_valid = f1_score(tfidf_result_valid, y_test)

print("f1 score train : ", tfidf_f1_sco_train)
print("f1 score valid : ", tfidf_f1_sco_valid)

f1 score train :  0.9445568595710239
f1 score valid :  0.7421276595744681


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
